In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
import fasttext
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score

import sys
sys.path.append('DialogueActClassification')

from dialogue_classification.dataset_utils import load_swda_corpus_data
from dialogue_classification.train_test_split import train_set_idx, valid_set_idx, test_set_idx

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz

In [17]:
!gzip -dk cc.en.300.bin.gz

## Prepare data

In [3]:
talks, talk_names, tag_indices, tag_occurances = load_swda_corpus_data('swda/swda/swda')

Loading SwDA Corpus...

Found 43 different utterance tags.

Loaded SwDA Corpus.


In [4]:
len(talks), len(tag_indices)

(1155, 43)

In [5]:
import re

def preprocess(text):
    text = text.lower()
    text = re.sub("[,']+", '', text)
    text = re.sub('\.', ' . ', text)
    text = re.sub('\?', ' ? ', text)
    text = re.sub('!', ' ! ', text)
    text = re.sub('\s+', ' ', text)
    return text

In [6]:
texts, labels = [], []
for text, label in talks:
    assert len(text) == len(label)
    text = [' '.join(utterance) for utterance in text]
    texts.append(text)
    labels.append(label)

In [7]:
texts = [[preprocess(utt) for utt in text] for text in texts]

In [8]:
train_idx = np.isin(talk_names, train_set_idx)
valid_idx = np.isin(talk_names, valid_set_idx)
test_idx = np.isin(talk_names, test_set_idx)

In [9]:
texts, labels = np.array(texts), np.array(labels)

train_texts, train_labels = texts[train_idx], labels[train_idx]
valid_texts, valid_labels = texts[valid_idx], labels[valid_idx]
test_texts, test_labels = texts[test_idx], labels[test_idx]

/var/folders/pk/0hw9m8496w7gxnl1pfsssfdddh310s/T/ipykernel_33254/1925198038.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  texts, labels = np.array(texts), np.array(labels)


In [10]:
len(train_texts), len(valid_texts), len(test_texts)

(1003, 112, 19)

In [11]:
# flatten
train_texts = sum(train_texts.tolist(), [])
valid_texts = sum(valid_texts.tolist(), [])
test_texts = sum(test_texts.tolist(), [])

train_labels = np.array(sum(train_labels.tolist(), []))
valid_labels = np.array(sum(valid_labels.tolist(), []))
test_labels = np.array(sum(test_labels.tolist(), []))

In [12]:
train_texts[:10]

['so ive been concerned about crime lately . ',
 'uh-huh . ',
 'uh its really scary to listen to the news every night and --',
 'uh-huh . ',
 '-- to hear about all the problems . ',
 'i wondered if you were taking any special precautions in your neighborhood ? ',
 'well i i think we have a neighborhood watch <laughter> . ',
 'uh-huh . ',
 'i think . ',
 '<laughter> . ']

In [13]:
train_labels[:10]

array([38, 11, 39, 11,  1, 37, 25, 11, 38, 42])

## Fasttext embeddings

In [14]:
fasttext_model = fasttext.load_model("cc.en.300.bin")

In [15]:
from tqdm import tqdm

X_train = [fasttext_model.get_sentence_vector(text) for text in tqdm(train_texts)]
X_valid = [fasttext_model.get_sentence_vector(text) for text in tqdm(valid_texts)]
X_test = [fasttext_model.get_sentence_vector(text) for text in tqdm(test_texts)]

X_train = np.stack(X_train)
X_valid = np.stack(X_valid)
X_test = np.stack(X_test)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4514/4514 [00:00<00:00, 107568.25it/s]


In [16]:
X_train.shape

(193325, 300)

## Train & Eval

In [17]:
model = LGBMClassifier(n_estimators=1000, objective='multiclass', n_jobs=-1, random_state=42, learning_rate=0.02)

model.fit(
    X_train, 
    train_labels, 
    eval_metric='multi_logloss', 
    eval_set=[(X_valid, valid_labels)],
    early_stopping_rounds=50, 
    verbose=50,
)

/Users/e.trufanova/.pyenv/versions/3.8.6/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/e.trufanova/.pyenv/versions/3.8.6/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[50]	valid_0's multi_logloss: 1.24505
[100]	valid_0's multi_logloss: 1.09023
[150]	valid_0's multi_logloss: 1.04074
[200]	valid_0's multi_logloss: 1.02112
[250]	valid_0's multi_logloss: 1.01343
[300]	valid_0's multi_logloss: 1.01141
[350]	valid_0's multi_logloss: 1.01238


LGBMClassifier(learning_rate=0.02, n_estimators=1000, objective='multiclass',
               random_state=42)

In [18]:
f1_score(test_labels, model.predict(X_test), average='micro')

0.6657066902968543